# Projeto 2 - Ciência dos Dados

**Nome: Breno Marti**

**Nome: João Pedro Chacon Ruiz**

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emojis

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
import numpy as np
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [3]:
'''
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])
'''

"\n#Dados de autenticação do twitter:\n\n#Coloque aqui o identificador da conta no twitter: @fulano\n\n#leitura do arquivo no formato JSON\nwith open('auth.pass') as fp:    \n    data = json.load(fp)\n\n#Configurando a biblioteca. Não modificar\nauth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])\nauth.set_access_token(data['access_token'], data['access_token_secret'])\n"

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [4]:
#Produto escolhido:
produto = 'Bugatti'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
'''
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():
    if (not msg.retweeted) and ('RT' not in msg.full_text):
        msgs.append(msg.full_text.lower())
        i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs'
'''

'\n#Cria um objeto para a captura\napi = tweepy.API(auth)\n\n#Inicia a captura, para mais detalhes: ver a documentação do tweepy\ni = 1\nmsgs = []\nfor msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():\n    if (not msg.retweeted) and (\'RT\' not in msg.full_text):\n        msgs.append(msg.full_text.lower())\n        i += 1\n    if i > n:\n        break\n\n#Embaralhando as mensagens para reduzir um possível viés\nshuffle(msgs\'\n'

Salvando os dados em uma planilha Excel:

In [6]:
'''
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()
'''

"\n#Verifica se o arquivo não existe para não substituir um conjunto pronto\nif not os.path.isfile('./{0}.xlsx'.format(produto)):\n    \n    #Abre o arquivo para escrita\n    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))\n\n    #divide o conjunto de mensagens em duas planilhas\n    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})\n    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)\n\n    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})\n    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)\n\n    #fecha o arquivo\n    writer.save()\n"

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [7]:
raw = pd.read_excel('Bugatti_treino.xlsx')

In [8]:
relevante_treino_raw = raw[raw['Relevância'] == 1]

In [9]:
nao_relevante_treino_raw = raw[raw["Relevância"] == 0]

#### Funções de limpeza de texto:


In [10]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[#@,!\()-.€":?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed   

In [11]:
import emojis

In [12]:
def transform_emojis(text):
    clean = emojis.decode(text)
    return clean

#### Limpando e transformando os dataframes em séries para a análise:

In [13]:
l = ['\n','https','//t','co/']

In [14]:
list1 = []
for tweet in relevante_treino_raw['Treinamento']:
    texto = transform_emojis(tweet)
    list1.append(cleanup(texto))
    
relevante_treino = pd.Series(list1)

Nota do professor: explicar como o código abaixo funciona.

In [15]:
relevante_treino = relevante_treino.str.replace('|'.join(l),'',regex=True).str.strip()
relevante_treino

0     mds cade a koenigsegg pra bater esses 500km/h ...
1     bugatti bate recorde de velocidade com um carr...
2     depois de bater o próprio recorde de 490 km/h ...
3     podem não ser os mais bonitos  mas os carros d...
4     parece que teremos o bugatti chiron super spor...
5     bugatti descobriu o quão rápido o chiron é  fl...
6     bugatti chiron com velocidade final de 490 km/...
7      quase 500 kph em um carro  absurdo    cahutq6joi
8     tô pesquisando uns carros luxuosos aqui e perc...
9     após 490 km/h do chiron  bugatti anuncia que n...
10    bugatti chiron super sport 300   versão  civil...
11    bugatti ultrapassa a marca dos 480 km/h – e de...
12    bugatti quebra o recorde de velocidade com um ...
13    bugatti chiron chega a 490 km/h e bate recorde...
14    carro mais rápido do mundo  nova bugatti terá ...
15    williamenb outra coisa  porque aquele bugatti ...
16    wow  a bugatti chiron chegou a 490 km/h    iss...
17    bugatti chiron  levemente modificada  cheg

In [16]:
list2 = []
for tweet in nao_relevante_treino_raw['Treinamento']:
    texto = transform_emojis(tweet)
    list2.append(cleanup(texto))
    
nao_relevante_treino = pd.Series(list2)

In [17]:
nao_relevante_treino = nao_relevante_treino.str.replace('|'.join(l),'',regex=True).str.strip()
nao_relevante_treino

0      talla 42 eu  bugatti 3 11382e 11  botas clasic...
1      talla 40 eu  bugatti 3 11384e 11  botas clasic...
2      carolinaazered2  junior__bugatti com o bumbum ...
3                fuga de bugatti com ela depois das três
4      bugatti verón supersport motor triturbo 8 0 l ...
5      el nuevo récord de velocidad del bugatti chiro...
6      se continuar esse calor vai todo mundo derrete...
7                                 lleobertodo menino bom
8      gente vamos jogar stop  não vale olhar na inte...
9      aberto até de madrugada  bugatti chiron chega ...
10     gente vamos jogar stop  não vale olhar na inte...
11     luaracaastro desse jeito  daqui a pouco volta ...
12                    jenniferbraaga uma delicinha kkkkk
13                              paloma_souza18 foi ontem
14     ballerichh claro q  lembro alugamos um  bugatt...
15     mas eu conseguiria trazer um suco gástrico esp...
16              ofereçam me um bugatti  não peço mais xd
17     iamthebrunao tomara q nã

In [18]:
list3 = []
for tweet in raw['Treinamento']:
    texto = transform_emojis(tweet)
    list3.append(cleanup(texto))
    
tudo = pd.Series(list3)

In [19]:
tudo = tudo.str.replace('|'.join(l),'',regex=True).str.strip()
tudo

0      talla 42 eu  bugatti 3 11382e 11  botas clasic...
1      talla 40 eu  bugatti 3 11384e 11  botas clasic...
2      carolinaazered2  junior__bugatti com o bumbum ...
3      mds cade a koenigsegg pra bater esses 500km/h ...
4      bugatti bate recorde de velocidade com um carr...
5                fuga de bugatti com ela depois das três
6      bugatti verón supersport motor triturbo 8 0 l ...
7      el nuevo récord de velocidad del bugatti chiro...
8      se continuar esse calor vai todo mundo derrete...
9      depois de bater o próprio recorde de 490 km/h ...
10                                lleobertodo menino bom
11     gente vamos jogar stop  não vale olhar na inte...
12     podem não ser os mais bonitos  mas os carros d...
13     aberto até de madrugada  bugatti chiron chega ...
14     gente vamos jogar stop  não vale olhar na inte...
15     luaracaastro desse jeito  daqui a pouco volta ...
16                    jenniferbraaga uma delicinha kkkkk
17                             

#### Contando o número de aparições de cada palavra :

In [20]:
list4 = []
for tweet in tudo:
    list4 = list4 + tweet.split()

universo = pd.Series(list4)
universo.value_counts()

bugatti             223
de                  128
a                   124
o                   119
que                  75
um                   74
e                    69
do                   67
chiron               65
não                  62
na                   55
km/h                 46
carro                46
nome                 40
490                  40
eu                   39
é                    39
com                  37
em                   35
mais                 33
primeira             33
seu                  30
letra                26
para                 26
gente                26
por                  25
velocidade           24
da                   24
bananaobjeto         23
recorde              23
                   ... 
deitar                1
velho                 1
jenniferbraaga        1
tenta                 1
kkkkkkkk              1
more                  1
impressionante        1
exploding_head        1
derreter              1
y8wid57fjp            1
brumadinhobebida

In [21]:
list5 = []
for tweet in relevante_treino:
    list5 = list5 + tweet.split()

palavras_relevantes = pd.Series(list5)
palavras_relevantes.value_counts()

bugatti         76
o               55
a               53
de              52
chiron          45
km/h            38
490             31
que             31
carro           27
um              26
do              23
em              22
velocidade      20
mais            20
recorde         19
e               17
com             15
é               15
não             14
da              13
300             11
mph             10
sport           10
os               9
super            8
mundial          8
chega            8
rápido           7
por              7
500              7
                ..
distintas        1
absurdamente     1
cima             1
600              1
ye7je7pxvg       1
83avneux8y       1
vendido          1
saídas           1
andy             1
barrier          1
tomando          1
primeira         1
teria            1
sem              1
conhecidos       1
insano           1
qse              1
out              1
parabéns         1
pqp              1
momento          1
foda        

In [22]:
list6 = []
for tweet in nao_relevante_treino:
    list6 = list6 + tweet.split()

palavras_nao_relevantes = pd.Series(list6)
palavras_nao_relevantes.value_counts()

bugatti             147
de                   76
a                    71
o                    64
e                    52
na                   49
um                   48
não                  48
que                  44
do                   44
nome                 39
primeira             32
eu                   32
seu                  29
letra                26
gente                25
é                    24
bananaobjeto         23
vale                 22
olhar                22
com                  22
vamos                22
jogar                22
stop                 21
respondanome         21
brancofruta          21
chiron               20
bugatticor           20
para                 19
carro                19
                   ... 
botafogo              1
graças                1
5e26bo2s25            1
pqp                   1
mundial               1
xkgglqwab4            1
bilionários           1
oi                    1
briguento             1
produto               1
vem             

#### Frequência relativa de cada palavra:

In [23]:
universo.value_counts(True)

bugatti             0.044511
de                  0.025549
a                   0.024750
o                   0.023752
que                 0.014970
um                  0.014770
e                   0.013772
do                  0.013373
chiron              0.012974
não                 0.012375
na                  0.010978
km/h                0.009182
carro               0.009182
nome                0.007984
490                 0.007984
eu                  0.007784
é                   0.007784
com                 0.007385
em                  0.006986
mais                0.006587
primeira            0.006587
seu                 0.005988
letra               0.005190
para                0.005190
gente               0.005190
por                 0.004990
velocidade          0.004790
da                  0.004790
bananaobjeto        0.004591
recorde             0.004591
                      ...   
deitar              0.000200
velho               0.000200
jenniferbraaga      0.000200
tenta         

In [24]:
palavras_relevantes.value_counts(True)

bugatti         0.051213
o               0.037062
a               0.035714
de              0.035040
chiron          0.030323
km/h            0.025606
490             0.020889
que             0.020889
carro           0.018194
um              0.017520
do              0.015499
em              0.014825
velocidade      0.013477
mais            0.013477
recorde         0.012803
e               0.011456
com             0.010108
é               0.010108
não             0.009434
da              0.008760
300             0.007412
mph             0.006739
sport           0.006739
os              0.006065
super           0.005391
mundial         0.005391
chega           0.005391
rápido          0.004717
por             0.004717
500             0.004717
                  ...   
distintas       0.000674
absurdamente    0.000674
cima            0.000674
600             0.000674
ye7je7pxvg      0.000674
83avneux8y      0.000674
vendido         0.000674
saídas          0.000674
andy            0.000674


In [25]:
palavras_nao_relevantes.value_counts(True)

bugatti             0.041690
de                  0.021554
a                   0.020136
o                   0.018151
e                   0.014748
na                  0.013897
um                  0.013613
não                 0.013613
que                 0.012479
do                  0.012479
nome                0.011061
primeira            0.009075
eu                  0.009075
seu                 0.008225
letra               0.007374
gente               0.007090
é                   0.006807
bananaobjeto        0.006523
vale                0.006239
olhar               0.006239
com                 0.006239
vamos               0.006239
jogar               0.006239
stop                0.005956
respondanome        0.005956
brancofruta         0.005956
chiron              0.005672
bugatticor          0.005672
para                0.005389
carro               0.005389
                      ...   
botafogo            0.000284
graças              0.000284
5e26bo2s25          0.000284
pqp           

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [26]:
P_relevante = palavras_relevantes.size/universo.size
P_relevante

0.2962075848303393

In [27]:
P_nao_relevante = palavras_nao_relevantes.size/universo.size
P_nao_relevante

0.7037924151696606

In [28]:
raw2 = pd.read_excel('Bugatti_teste.xlsx')
relevante_teste_raw2 = raw2[raw2['Relevância'] == 1]
nao_relevante_teste_raw2 = raw2[raw2["Relevância"] == 0]

lista1 = []
for tweet in relevante_teste_raw2['Teste']:
    texto = transform_emojis(tweet)
    lista1.append(cleanup(texto))
    
relevante_teste = pd.Series(lista1)
relevante_teste = relevante_teste.str.replace('|'.join(l),'',regex=True).str.strip()

lista2 = []
for tweet in nao_relevante_teste_raw2['Teste']:
    texto = transform_emojis(tweet)
    lista2.append(cleanup(texto))
    
nao_relevante_teste = pd.Series(lista2)
nao_relevante_teste = nao_relevante_teste.str.replace('|'.join(l),'',regex=True).str.strip()

lista3 = []
for tweet in raw2['Teste']:
    texto = transform_emojis(tweet)
    lista3.append(cleanup(texto))
    
testudo = pd.Series(lista3)
testudo = testudo.str.replace('|'.join(l),'',regex=True).str.strip()

In [29]:
lista4 = []
for tweet in testudo:
    lista4 = lista4 + tweet.split()

universo_teste = pd.Series(lista4)
universo_teste.value_counts()

lista5 = []
for tweet in relevante_teste:
    lista5 = lista5 + tweet.split()

palavras_relevantes_teste = pd.Series(lista5)
P_R_T_R = palavras_relevantes_teste.value_counts(True)

lista6 = []
for tweet in nao_relevante_teste:
    lista6 = lista6 + tweet.split()

palavras_nao_relevantes_teste = pd.Series(lista6)
P_N_R_T_R = palavras_nao_relevantes_teste.value_counts(True)

### Montando o Classificardor Naive Bayes

#### Probabilidades de ser relevante ou não relevante

In [67]:
palavras_relevantes_teste

0              dirigida
1                  pelo
2                piloto
3                  andy
4               wallace
5              vencedor
6                    do
7                grande
8                prêmio
9                    de
10                   le
11                 mans
12                   em
13                 1988
14                  uma
15              bugatti
16               chiron
17           “levemente
18          modificada”
19               chegou
20                    a
21      impressionantes
22                  490
23                  484
24                 km/h
25           0lnmllrds9
26              bugatti
27               chiron
28                chega
29                    a
             ...       
1346                ver
1347             quanto
1348              tempo
1349                vai
1350              levar
1351                pra
1352         koenigsegg
1353                 se
1354         pronunciar
1355                  e
1356            

In [66]:
palavras_nao_relevantes_teste

0       femalelionisa
1             sqnpaty
2                 sai
3                 dai
4              feiosa
5                  eu
6          represento
7                kkkk
8               avião
9              ahahah
10         989k28qply
11         kethllyn31
12            bugatti
13                  e
14                uma
15              marca
16                 de
17              carro
18                não
19                  o
20              carro
21              geral
22                 me
23            olhando
24                 no
25         samuquinha
26                 só
27                 pq
28            cheguei
29                com
            ...      
2010              que
2011                é
2012             pior
2013           alguém
2014        acreditar
2015              que
2016                4
2017         salários
2018         incomoda
2019       bilionário
2020               ou
2021            achar
2022              que
2023               dá
2024      

In [30]:
palavras_teste = palavras_relevantes_teste.size + palavras_nao_relevantes_teste.size
palavras_teste

3416

In [40]:
Pt_relevante = relevante_teste.size / testudo.size
Pt_relevante

0.38

In [41]:
Ptn_relevante = nao_relevante_teste.size / testudo.size
Ptn_relevante

0.62

### Como o classificador funcionará:

Todos os tweets presentes na planilha de testes serão, individualmente, analisados pelo classificador e catalogados, em um loop. Depois de todo esse processo, os resultados de cada tweet analisado, que podem ser ou "Relevante" ou "Irrelevante", pelo menos neste primeiro momento, serão adicionados a uma lista com o fim de serem comparados com a análise feita à mão.

In [159]:
def Bayes(relevancia,serie1,serie2):
    #Criando o loop para o cálculo da probabilidade de ser relevante:
    i = 0 
    lista_resultados_numericos = []

    #selecionando o tweet
    for tweets in testudo:
        
        #transformando em lista de str
        tweet = testudo[i]
        tweetr = tweet.split()
        
        # Calculando a probabilidade deste tweet ser relevante dado as suas palavras
        log_probabilidade = 0
        for palavra in tweetr:
        
            #Não deixa que palavras não pertencentes à série bugem o código
            if palavra not in serie1:
                log_probabilidade += math.log(1/3416, 10)
            else:
                log_probabilidade += math.log(serie2[palavra] + 1/3416, 10)
        
        #Guardando os resultados e resetando a variável probabilidade    
        lista_resultados_numericos.append(log_probabilidade + math.log(relevancia, 10))
        log_probabilidade = 0
        i += 1

    return lista_resultados_numericos

In [171]:
tweetr

['fuga', 'de', 'bugatti', 'com', 'ela', 'dps', 'das', '3']

In [161]:
lista_resultados_numericos_relevantes = Bayes(Pt_relevante, palavras_relevantes_teste, P_R_T_R)
lista_resultados_numericos_relevantes 

[-28.68835929951893,
 -11.02076998943409,
 -35.75539502355286,
 -49.889466471620715,
 -92.2916808158243,
 -46.35594860960375,
 -56.95650219565464,
 -35.75539502355286,
 -56.95650219565464,
 -46.35594860960375,
 -78.15760936775644,
 -32.22187716153589,
 -145.29444874607876,
 -46.35594860960375,
 -32.22187716153589,
 -32.22187716153589,
 -102.89223440187519,
 -28.68835929951893,
 -53.42298433363768,
 -71.09057364372251,
 -42.822430747586786,
 -78.15760936775644,
 -60.49002005767161,
 -92.2916808158243,
 -85.22464509179036,
 -28.68835929951893,
 -74.62409150573947,
 -49.889466471620715,
 -39.28891288556982,
 -117.02630584994304,
 -18.087805713468025,
 -162.96203805616358,
 -46.35594860960375,
 -42.822430747586786,
 -124.09334157397697,
 -85.22464509179036,
 -71.09057364372251,
 -14.554287851451058,
 -28.68835929951893,
 -64.02353791968858,
 -39.28891288556982,
 -106.42575226389215,
 -25.15484143750196,
 -53.42298433363768,
 -32.22187716153589,
 -85.22464509179036,
 -42.822430747586786,
 -

In [162]:
lista_resultados_numericos_nrelevantes = Bayes(Ptn_relevante, palavras_nao_relevantes_teste, P_N_R_T_R)
lista_resultados_numericos_nrelevantes

[-28.475751206637486,
 -10.808161896552646,
 -35.54278693067142,
 -49.676858378739276,
 -92.07907272294284,
 -46.14334051672231,
 -56.743894102773204,
 -35.54278693067142,
 -56.743894102773204,
 -46.14334051672231,
 -77.94500127487498,
 -32.009269068654454,
 -145.0818406531973,
 -46.14334051672231,
 -32.009269068654454,
 -32.009269068654454,
 -102.67962630899373,
 -28.475751206637486,
 -53.21037624075624,
 -70.87796555084105,
 -42.60982265470535,
 -77.94500127487498,
 -60.27741196479017,
 -92.07907272294284,
 -85.01203699890891,
 -28.475751206637486,
 -74.41148341285802,
 -49.676858378739276,
 -39.07630479268838,
 -116.81369775706159,
 -17.875197620586583,
 -162.74942996328213,
 -46.14334051672231,
 -42.60982265470535,
 -123.88073348109552,
 -85.01203699890891,
 -70.87796555084105,
 -14.341679758569613,
 -28.475751206637486,
 -63.81092982680713,
 -39.07630479268838,
 -106.2131441710107,
 -24.94223334462052,
 -53.21037624075624,
 -32.009269068654454,
 -85.01203699890891,
 -42.6098226547

In [164]:
i=0
lista_resultados = []
for tweet in testudo:
    
    if lista_resultados_numericos_relevantes[i] > lista_resultados_numericos_nrelevantes[i]:
           lista_resultados.append("Relevante")
            
    if lista_resultados_numericos_relevantes[i] == lista_resultados_numericos_nrelevantes[i]:
        lista_resultados.append("Neutro")
        
    if lista_resultados_numericos_relevantes[i] < lista_resultados_numericos_nrelevantes[i]:
        lista_resultados.append("Não relevante")
        
    i += 1
    
#lista_resultados

In [170]:

i=0
cu=[]

for tweet in testudo:
    cu.append(lista_resultados_numericos_relevantes[i] / lista_resultados_numericos_nrelevantes[i])
cu

[1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856596346,
 1.0074662856

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**